In [57]:
import requests
import json
import os

from requests.auth import HTTPBasicAuth

elastic_password = "KCXLG4FSMBvROysWexMJ"
tmdb_api_key = "337558ef623d231ce36cd7d3456665a0"

tmdb_api = requests.Session()
tmdb_api.params={'api_key': tmdb_api_key}

def flatten(l):
    [item for sublist in l for item in sublist]

def simplerExplain(explainJson, depth=0):
    result = " " * (depth * 2) + "%s, %s\n" % (explainJson['value'], explainJson['description'])
    #print json.dumps(explainJson, indent=True)
    if 'details' in explainJson:
        for detail in explainJson['details']:
            result += simplerExplain(detail, depth=depth+1)
    return result


In [101]:

## 3.2.2 Indexing TMDB Movies

movieIds = []
numMoviesToGrab = 5000
numPages = int(numMoviesToGrab / 20)

for page in range(1, numPages + 1):
    httpResp = tmdb_api.get('https://api.themoviedb.org/3/movie/top_rated', params={'page': page})  #(1)
    jsonResponse = json.loads(httpResp.text) #(2)
    movies = jsonResponse['results']
    for movie in movies:
        if (movie['id'] not in [9549]):
            movieIds.append(movie['id'])
print(len(movieIds))

4999


In [103]:
movieDict = {}
for movieId in movieIds:
    httpResp = tmdb_api.get("https://api.themoviedb.org/3/movie/%s" % movieId)
    movie = json.loads(httpResp.text)
    movieDict[movieId] = movie

In [291]:
resp = requests.delete("http://localhost:9200/tmdb")
print(resp.status_code)

404


In [292]:
settings = {
    "settings": {"number_of_shards": 1}
}
resp = requests.put("http://localhost:9200/tmdb", data=json.dumps(settings), headers={"content-type": "application/json"})
print(resp.status_code)

200


In [293]:
print("Indexing %i movies" % len(movieDict.keys()))
bulkMovies = ""
for id, movie in movieDict.items():
    addCmd = {"index": {"_index": "tmdb", "_type": "movie", "_id": movie["id"]}}
    esDoc  = {"title": movie['title'], 'overview': movie['overview'], 'tagline': movie['tagline']}
    bulkMovies += json.dumps(addCmd) + "\n" + json.dumps(esDoc) + "\n"
requests.post(f"http://localhost:9200/_bulk", data=bulkMovies, headers={"content-type": "application/json"})


Indexing 4999 movies


<Response [200]>

In [294]:
usersSearch = 'basketball with cartoon aliens'
search = {
    'query': {
        'multi_match': {
            'query': usersSearch,  #User's query
            'fields': ['title^10', 'overview'],
        }
    },
    'size': '100',
    'explain': True
}

httpResp = requests.get('http://localhost:9200/tmdb/movie/_search', data=json.dumps(search), headers={"content-type": "application/json"})
searchHits = json.loads(httpResp.text)['hits']
print("Num\tRelevance Score\t\tMovie Title\t\tOverview")
for idx, hit in enumerate(searchHits['hits']):
        print("%s\t%s\t\t%s\t\t%s" % (idx + 1, hit['_score'], hit['_source']['title'], len(hit['_source']['overview'])))

Num	Relevance Score		Movie Title		Overview
1	110.91062		Aliens		354
2	87.18218		Love & Basketball		420
3	75.08492		The Basketball Diaries		119
4	48.600666		Weathering with You		548
5	48.600666		Dances with Wolves		239
6	48.600666		Summer with Monika		217
7	48.600666		With Every Heartbeat		247
8	48.600666		Playing with Fire		97
9	48.600666		Friends with Benefits		250
10	42.678642		Gone with the Wind		159
11	42.678642		My Dinner with Andre		133
12	42.678642		Interview with the Vampire		113
13	42.678642		From Russia with Love		441
14	42.678642		The Trouble with Harry		433
15	42.678642		Trouble with the Curve		245
16	42.678642		Fighting with My Family		447
17	42.678642		My Week with Marilyn		319
18	42.678642		Just Go with It		340
19	38.043076		The House with Laughing Windows		178
20	38.043076		Die Hard: With a Vengeance		382
21	38.043076		The Kid with a Bike		95
22	38.043076		A Room with a View		324
23	38.043076		Riding in Cars with Boys		147
24	38.043076		Girl with a Pearl Earring		364
25

In [295]:
search = {
   'query': {
        'multi_match': {
            'query': usersSearch,  #User's query
            'fields': ['title^10', 'overview']
        }
    }
}
httpResp = requests.get('http://localhost:9200' +
			    '/tmdb/movie/_validate/query?explain',
			     data=json.dumps(search), headers={"content-type": "application/json"})
print(json.loads(httpResp.text))

{'_shards': {'total': 1, 'successful': 1, 'failed': 0}, 'valid': True, 'explanations': [{'index': 'tmdb', 'valid': True, 'explanation': '+((overview:basketball overview:with overview:cartoon overview:aliens) | (title:basketball title:with title:cartoon title:aliens)^10.0) #*:*'}]}


In [296]:
# Inner Layer of the Onion -- Why did the search engine consider these movies matches? Two sides to this
# (1) What tokens are placed in the search engine?
# (2) What did the search engine attempt to match exactly?

# Explain of what's happening when we construct these terms

#resp = requests.get(elasticSearchUrl + "/tmdb/_mapping/movie/field/title?format=yaml'
resp = requests.get('http://localhost:9200/tmdb/_analyze',
                    data='{"text":"Fire with Fire"}',headers={"content-type": "application/json"})
print(resp.text)

{"tokens":[{"token":"fire","start_offset":0,"end_offset":4,"type":"<ALPHANUM>","position":0},{"token":"with","start_offset":5,"end_offset":9,"type":"<ALPHANUM>","position":1},{"token":"fire","start_offset":10,"end_offset":14,"type":"<ALPHANUM>","position":2}]}


In [324]:
from time import sleep
# DELETE AND RECREATE THE INDEX WITH ENGLISH ANALYZERS
requests.delete('http://localhost:9200/tmdb', headers={"content-type": "application/json"})

<Response [200]>

In [325]:
settings = {
    'settings': {
        "number_of_shards": 1,
    },
    'mappings': {
        'movie': {
            'properties': {
                'title': {
                    'type': 'text',
                    'analyzer': 'english'
                },
                'overview': {
                    'type': 'text',
                    'analyzer': 'english'
                }
            }
        }
       }
}

resp = requests.put('http://localhost:9200/tmdb/?include_type_name=true', data=json.dumps(settings), headers={"content-type": "application/json"})
print(resp.text)
sleep(1)

{"acknowledged":true,"shards_acknowledged":true,"index":"tmdb"}


In [326]:
# Inspecting the mappings
resp = requests.get('http://localhost:9200/tmdb/_mappings?format=yaml')
print(resp.text)

---
tmdb:
  mappings:
    properties:
      overview:
        type: "text"
        analyzer: "english"
      title:
        type: "text"
        analyzer: "english"



In [327]:
# Reanalyze the string

resp = requests.get('http://localhost:9200/tmdb/_analyze',
                    data='{"text":"Fire with Fire"}',headers={"content-type": "application/json"})
print(resp.text)

{"tokens":[{"token":"fire","start_offset":0,"end_offset":4,"type":"<ALPHANUM>","position":0},{"token":"with","start_offset":5,"end_offset":9,"type":"<ALPHANUM>","position":1},{"token":"fire","start_offset":10,"end_offset":14,"type":"<ALPHANUM>","position":2}]}


In [328]:
# Reindex
resp = requests.post('http://localhost:9200/_bulk', data=bulkMovies, headers={"content-type": "application/json"})
print(resp.status_code)
resp = requests.get('http://localhost:9200/tmdb/_refresh')
print(resp.status_code)
print(resp.text)

200
200
{"_shards":{"total":2,"successful":1,"failed":0}}


In [329]:
search = {
    'query': {
        'multi_match': {
            'query': usersSearch,  #User's query
            'fields': ['title^10', 'overview'],
        }
    },
    'size': '100',
    'explain': True
}
httpResp = requests.get('http://localhost:9200/tmdb/movie/_search', data=json.dumps(search), headers={"content-type": "application/json"})

searchHits = json.loads(httpResp.text)['hits']

print("Num\tRelevance Score\t\tMovie Title\t\tOverview")
for idx, hit in enumerate(searchHits['hits']):
        print("%s\t%s\t\t%s\t\t%s" % (idx + 1, hit['_score'], hit['_source']['title'], len(hit['_source']['overview'])))


Num	Relevance Score		Movie Title		Overview
1	93.90564		Alien		420
2	93.90564		Aliens		354
3	79.4909		Love & Basketball		420
4	79.4909		The Basketball Diaries		119
5	54.97399		Scooby-Doo and the Alien Invaders		175
6	8.5429735		Don't Worry, He Won't Get Far on Foot		228
7	8.173914		The Way Back		246
8	7.984569		Uncle Drew		106
9	7.8472304		White Men Can't Jump		86
10	7.342078		He Got Game		113
11	7.003928		Glory Road		146
12	6.8980293		Air		202
13	6.7952843		Live Flesh		164
14	6.7553997		Attraction		174
15	6.5987115		Coach Carter		171
16	6.5987115		Champions		215
17	6.065633		District 9		466
18	6.020622		Invasion of the Body Snatchers		114
19	5.918814		The Day the Earth Stood Still		106
20	5.918814		No One Will Save You		89
21	5.8203936		Mysterious Skin		128
22	5.8203936		Cocoon		128
23	5.755653		Pacific Rim		358
24	5.692946		High School Musical		343
25	5.6330543		The Thing from Another World		125
26	5.622782		Hustle		291
27	5.5723925		Edge of Tomorrow		371
28	5.5723925		Independence Da

In [330]:
search['explain'] = True
httpResp = requests.get("http://localhost:9200" + '/tmdb/movie/_search', data=json.dumps(search), headers={"content-type": "application/json"})
jsonResp = json.loads(httpResp.text)
print(json.dumps(jsonResp['hits']['hits'][0]['_explanation'], indent=True))
print("Explain for %s" % jsonResp['hits']['hits'][0]['_source']['title'])
print(simplerExplain(jsonResp['hits']['hits'][0]['_explanation']))
print("Explain for %s" % jsonResp['hits']['hits'][1]['_source']['title'])
print(simplerExplain(jsonResp['hits']['hits'][1]['_explanation']))
print("Explain for %s" % jsonResp['hits']['hits'][2]['_source']['title'])
print(simplerExplain(jsonResp['hits']['hits'][2]['_explanation']))
print("Explain for %s" % jsonResp['hits']['hits'][3]['_source']['title'])
print(simplerExplain(jsonResp['hits']['hits'][3]['_explanation']))
print("Explain for %s" % jsonResp['hits']['hits'][10]['_source']['title'])
print(simplerExplain(jsonResp['hits']['hits'][10]['_explanation']))

{
 "value": 93.90564,
 "description": "max of:",
 "details": [
  {
   "value": 3.7182212,
   "description": "sum of:",
   "details": [
    {
     "value": 3.7182212,
     "description": "weight(overview:alien in 162) [PerFieldSimilarity], result of:",
     "details": [
      {
       "value": 3.7182212,
       "description": "score(freq=1.0), computed as boost * idf * tf from:",
       "details": [
        {
         "value": 2.2,
         "description": "boost",
         "details": []
        },
        {
         "value": 4.4829526,
         "description": "idf, computed as log(1 + (N - n + 0.5) / (n + 0.5)) from:",
         "details": [
          {
           "value": 56,
           "description": "n, number of documents containing term",
           "details": []
          },
          {
           "value": 4999,
           "description": "N, total number of documents with field",
           "details": []
          }
         ]
        },
        {
         "value": 0.3770061,
     

In [332]:
search = {
    'query': {
        'multi_match': {
            'query': usersSearch,  #User's query
            'fields': ['title^0.1', 'overview'],
        }
    },
    'size': '100',
    'explain': True
}
httpResp = requests.get('http://localhost:9200/tmdb/movie/_search', data=json.dumps(search), headers={"content-type": "application/json"})
searchHits = json.loads(httpResp.text)['hits']

print("Num\tRelevance Score\t\tMovie Title\t\tOverview")
for idx, hit in enumerate(searchHits['hits']):
        print("%s\t%s\t\t%s\t\t%s" % (idx + 1, hit['_score'], hit['_source']['title'], len(hit['_source']['overview'])))

Num	Relevance Score		Movie Title		Overview
1	8.5429735		Don't Worry, He Won't Get Far on Foot		228
2	8.173914		The Way Back		246
3	7.984569		Uncle Drew		106
4	7.8472304		White Men Can't Jump		86
5	7.462169		The Basketball Diaries		119
6	7.342078		He Got Game		113
7	7.149197		Love & Basketball		420
8	7.003928		Glory Road		146
9	6.8980293		Air		202
10	6.7952843		Live Flesh		164
11	6.7553997		Attraction		174
12	6.5987115		Coach Carter		171
13	6.5987115		Champions		215
14	6.065633		District 9		466
15	6.020622		Invasion of the Body Snatchers		114
16	5.918814		The Day the Earth Stood Still		106
17	5.918814		No One Will Save You		89
18	5.8203936		Mysterious Skin		128
19	5.8203936		Cocoon		128
20	5.755653		Pacific Rim		358
21	5.692946		High School Musical		343
22	5.6330543		The Thing from Another World		125
23	5.622782		Hustle		291
24	5.5723925		Edge of Tomorrow		371
25	5.5723925		Independence Day		391
26	5.5438356		Arrival		159
27	5.5438356		Avatar		175
28	5.48507		Star Trek IV: The Voyage Ho